In [1]:
from linked_list import LinkedList

class Stack(LinkedList):
    
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.head = self.tail = None
        else:
            self.tail = self.tail.prev
            self.tail.nex = None
        self.length -= 1
        return ret
    
# stack = Stack()
# for i in range(1, 4):
#     stack.push(i)
    
# top = stack.pop()
# print(stack.peek())

2


### 2. Infix and Postfix Notation

Define a function `tokenize()` that, given a string, uses the `str.split()` method to tokenize the expression.

In [2]:
def tokenize(expression):
    return expression.split()

print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


### 3. Processing an Operator

It is very important to perform the operation between the elements that was second to to and the top elements. If we do it the other way around we'll get the wrong result.

For example, in the `process_minus()` function we do:

`result = second_to_top - top # Correct`

and not

`result = top - second_to_top # Wrong`

In [3]:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)
    
    
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)
    
    
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)
    
    
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)
    
    
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

### 4. Evaluating Postfix Expressions

Here are the steps we need to follow to implement the `evaluate_postfix()` function.

Initialize an empty stack.

Tokenize the expression using the `tokenize()` function.

For each token, do:
    If the token is an operator, call the corresponding function to process it. For example, if we find a `+` we call the `process_plus()` function.
    Otherwise (the token is a number) and we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a `float` first.

Return the value that is left in the stack.

In [6]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            # The token is not an operator so it must be a number
            stack.push(float(token))
    return stack.pop()
                       

### Testing the implementation

When testing with other expressions we need to add spaces between at two tokens. For example `1 + 3` will work but `1+3` won't.

In [7]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


### 5. Operator Precedence in Infix Notation

The precedence dictionary is used to compare the precedence of two operators.

In [9]:
precedence = {"+": 1, "-": 1, "*": 2, "/": 2, "**": 3}

print(precedence["/"] < precedence["-"])
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

False
True
False
True


### 6. From Infix to Postfix

On this screen, we'll implement a function `infix_to_postfix()` that, given a string with an expression in infix notation, outputs a string with that expression written in postfix notation.

Here's an example of usage:


`infix = "10 + 3 * 5 / ( 16 - 4 )"`

postfix = `infix_to_postfix`(exp)

`print(postfix)`


10 3 5 * 16 4 - / +

In [10]:
# Opening parenthesis
def process_opening_parenthesis(stack):
    return stack.push("(")


In [11]:
# Closing parenthesis
def process_closing_parenthesis(stack, postfix):
    # Add tokens until we find the open bracket
    while stack.peek() != "(":
        postfix.append(stack.pop())
    # Remove the opening bracket
    stack.pop()
    

In [13]:
# Handling Operators
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)
    

In [14]:
# Handling Operands (any numbers)
def process_number(postfix, number):
    postfix.append(number)


### The Shunting-yard Algorithm

We now have all the pieces we need to implement the infix_to_postfix() function that converts an expression from infix notation to postfix notation.

1. We start by splitting the expression into tokens using the `tokenize()` function.

2. We initialize an empty stack.

3. We initialize and empty postfix token list.

4. Iterate over all tokens and for each of them:

    * If the token is `"("` we call the `process_opening_parenthesis()` function.
    
    * If the token is `")"` we call the `process_closing_parenthesis()` function.
    
    * If the token is an `operator` we call the `process_operator()` function.
    
    * Otherwise, the token is a `number` and we call the `process_number()` function.
    

5. After processing all tokens, we use a `while` loop to pop the remaining stack element into the postfix token list.

6. Use the `str.join()` method to convert the postfix token list into a string.

In [15]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in ("+", "-", "*", "/", "**"):
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    
    while len(stack) > 0:
        postfix.append(stack.pop())
        
    return " ".join(postfix)


###  Evaluating Infix expressions

In [19]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)


In [20]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0


### TODO

One of the limitations of our implementation is that it requires spaces to separate every part of the expression. For example, we can evaluate the expression `2 * ( 5 - 3 )`, but we can't evaluate `2 * (5 - 3)` or `2*(5 - 3)`. If you're interested in expanding this, you could think about improving the `tokenize()` method to make it more robust.

Another suggestion is to implement other operators — for example, the integer division `//`.

The `evaluate()` function that we implemented exists in Python as the `eval()` built-in function. The Python implementation can actually evaluate any string of Python code, not only expressions